In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import dgl
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from model_functions import MultiModal, Graph, Finger

# Turn to Tensor
train_graphs = [dgl.batch([g]) for g in train_graphs]
test_graphs = [dgl.batch([g]) for g in test_graphs]
train_y = torch.from_numpy(train_y).type(torch.long)
test_y = torch.from_numpy(test_y).type(torch.long)
if train_fingers is not None:
    train_fingers = torch.from_numpy(train_fingers).type(torch.float32)
if test_fingers is not None:
    test_fingers = torch.from_numpy(test_fingers).type(torch.float32)

############################################Parameters############################################
g_in_feats = molgraph[0].ndata['x'].shape[1]
g_hidden_size = [64, 32, 16]  
g_out_feats = 2
graph = Graph(g_in_feats, g_hidden_size, g_out_feats)

d_in_feats = 1012
d_hidden_size = 64
d_out_feats = 2
finger = Finger(d_in_feats, d_hidden_size, d_out_feats)

########################################### Train the Model ########################################
model = MultiFusion(g_in_feats, g_hidden_size, g_out_feats, d_in_feats, d_hidden_size, d_out_feats)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
model.train()

##########################################Batch######################################################
batch_size = 50
iter_train = int(len(train_graphs) / batch_size)
iter_test = int(len(test_graphs) / batch_size)

train_batches_graphs = []
train_batches = []
for i in range(iter_train):
    batch_graphs = train_graphs[i * batch_size:(i + 1) * batch_size]
    batch_targets = train_y[i * batch_size:(i + 1) * batch_size]
    batch = dgl.batch(batch_graphs)
    train_batches_graphs.append(batch)
    train_batches.append((batch, batch_targets))

train_labels_batch = []
for j in range(iter_train):
    label1 = []
    for i in range(batch_size):
        t = train_y[j * batch_size + i].numpy()
        label1.append(t)
        train_y_tensor = torch.tensor(label1, dtype=torch.long)
        train_y_tensor_squeezed = train_y_tensor.squeeze()
    train_labels_batch.append(train_y_tensor_squeezed)

train_batches_fingers = []
if train_fingers is not None:
    for j in range(iter_train):
        batches_fingers = []
        for i in range(batch_size):
            d = train_fingers[j * batch_size + i].numpy()
            batches_fingers.append(d)
            train_batches_finger = torch.tensor(batches_fingers)
            train_batches_finger_float = train_batches_finger.float()
        train_batches_fingers.append(train_batches_finger_float)

test_batches_graphs = []
test_batches = []
for i in range(iter_test):
    batch_graphs = test_graphs[i * batch_size:(i + 1) * batch_size]
    batch_targets = test_y[i * batch_size:(i + 1) * batch_size]
    batch = dgl.batch(batch_graphs)
    test_batches_graphs.append(batch)
    test_batches.append((batch, batch_targets))

test_labels_batch = []
for j in range(iter_test):
    label1 = []
    for i in range(batch_size):
        t = test_y[j * batch_size + i].numpy()
        label1.append(t)
        test_y_tensor = torch.tensor(label1, dtype=torch.long)
        test_y_tensor_squeezed = test_y_tensor.squeeze()
    test_labels_batch.append(test_y_tensor_squeezed)

test_batches_fingers = []
if test_fingers is not None:
    for j in range(iter_test):
        batches_fingers = []
        for i in range(batch_size):
            d = test_fingers[j * batch_size + i].numpy()
            batches_fingers.append(d)
            test_batches_finger = torch.tensor(batches_fingers)
            test_batches_finger_float = test_batches_finger.float()
        test_batches_fingers.append(test_batches_finger_float)